In [19]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import time

#URL（福島県会津若松市の賃貸住宅情報 検索結果の1ページ目）
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=020&bs=040&ta=07&sc=07202&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'

#データ取得
result = requests.get(url)
c = result.content

#HTMLを元に、オブジェクトを作る
soup = BeautifulSoup(c)

#物件リストの部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})

#ページ数を取得
body = soup.find("body")
pages = body.find_all("div",{'class':'pagination pagination_set-nav'})
pages_text = str(pages)
pages_split = pages_text.split('</a></li>\n</ol>')
pages_split0 = pages_split[0]
pages_split1 = pages_split0[-3:]
pages_split2 = pages_split1.replace('>','')
pages_split3 = int(pages_split2)

#URLを入れるリスト
urls = []

#1ページ目を格納
urls.append(url)

#2ページ目から最後のページまでを格納
for i in range(pages_split3-1):
    pg = str(i+2)
    url_page = url + '&pn=' + pg
    urls.append(url_page)
    
name = [] #マンション名
address = [] #住所
locations0 = [] #立地1つ目（最寄駅/徒歩~分）
locations1 = [] #立地2つ目（最寄駅/徒歩~分）
locations2 = [] #立地3つ目（最寄駅/徒歩~分）
age = [] #築年数
height = [] #建物高さ
floor = [] #階
rent = [] #賃料
admin = [] #管理費
others = [] #敷/礼/保証/敷引,償却
floor_plan = [] #間取り
area = [] #専有面積

#各ページで以下の動作をループ
for url in urls:
    #物件リストを切り出し
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)
    summary = soup.find("div",{'id':'js-bukkenList'})
    
    #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し
    cassetteitems = summary.find_all("div",{'class':'cassetteitem'})

    #各cassetteitemsに対し、以下の動作をループ
    for i in range(len(cassetteitems)):
        #各建物から売りに出ている部屋数を取得
        tbodies = cassetteitems[i].find_all('tbody')
        
        #マンション名取得
        subtitle = cassetteitems[i].find_all("div",{
            'class':'cassetteitem_content-title'})
        subtitle = str(subtitle)
        subtitle_rep = subtitle.replace(
            '[<div class="cassetteitem_content-title">', '')
        subtitle_rep2 = subtitle_rep.replace(
            '</div>]', '')

        #住所取得
        subaddress = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col1'})
        subaddress = str(subaddress)
        subaddress_rep = subaddress.replace(
            '[<li class="cassetteitem_detail-col1">', '')
        subaddress_rep2 = subaddress_rep.replace(
            '</li>]', '')
        
        #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
        for y in range(len(tbodies)):
            name.append(subtitle_rep2)
            address.append(subaddress_rep2)

        #立地を取得
        sublocations = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col2'})
        
        #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
        for x in sublocations:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        locations0.append(text)
                    elif i == 1:
                        locations1.append(text)
                    elif i == 2:
                        locations2.append(text)
                        
        #築年数と建物高さを取得
        tbodies = cassetteitems[i].find_all('tbody')
        col3 = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col3'})
        for x in col3:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        age.append(text)
                    else:
                        height.append(text)

    #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
    tables = summary.find_all('table')

    #各建物（table）に対して、売りに出ている部屋（row）を取得
    rows = []
    for i in range(len(tables)):
        rows.append(tables[i].find_all('tr'))

    #各部屋に対して、tableに入っているtext情報を取得し、dataリストに格納
    data = []
    for row in rows:
        for tr in row:
            cols = tr.find_all('td')
            for td in cols:
                text = td.find_all('li')
                if len(text) is not 0:
                    for v in text:
                        data.append(v.text)
                else:
                    data.append(td.text)

    #dataリストから、階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積を順番に取り出す
    index = 0
    for item in data:
        if '階' in item:
            floor.append(data[index])
            rent.append(data[index+1])
            admin.append(data[index+2])
            others.append(data[index+3])
            floor_plan.append(data[index+4])
            area.append(data[index+5])
        index +=1
    
    #プログラムを10秒間停止する（スクレイピングマナー）
    time.sleep(10)

#各リストをシリーズ化
name = Series(name)
address = Series(address)
locations0 = Series(locations0)
locations1 = Series(locations1)
locations2 = Series(locations2)
age = Series(age)
height = Series(height)
floor = Series(floor)
rent = Series(rent)
admin = Series(admin)
others = Series(others)
floor_plan = Series(floor_plan)
area = Series(area)

#各シリーズをデータフレーム化
suumo_df = pd.concat([name, address, locations0, locations1,
                      locations2, age, height,floor,rent,admin,others,floor_plan,area],axis=1)

#カラム名
suumo_df.columns=['マンション名','住所','立地1','立地2','立地3','築年数','建物高さ','階','賃料','管理費',
                  '敷/礼/保証/敷引,償却','間取り','専有面積']

#csvファイルとして保存
suumo_df.to_csv('suumo_adachi.csv', sep = '\t',encoding='utf-16')

['\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5万円', '1500円', '-', '5万円', '1K', '32.43m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.1万円', '1500円', '-', '5.1万円', '1K', '32.43m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.2万円', '1500円', '-', '5.2万円', '1K', '32.23m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.3万円', '1500円', '-', '5.3万円', '1K', '32.23m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '5.7万円', '1500円', '-', '5.7万円', '1K', '40.94m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '5.7万円', '1500円', '-', '5.7万円', '1K', '40.94m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '5.9万円', '1500円', '-', '5.9万円', '1K', '40.74m

['\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '4.5万円', '-', '-', '-', 'ワンルーム', '26m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t3階', '4.5万円', '-', '-', '4.5万円', 'ワンルーム', '26m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1-2階', '3.7万円', '-', '-', '-', '3DK', '52.99m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t3階', '7.2万円', '-', '14.4万円', '-', '3LDK', '75.4m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.8万円', '-', '3.8万円', '-', '4DK', '59.94m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.8万円', '-', '-', '-', '2DK', '39.83m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '4.7万円', '-', '-', '-', '3DK', '51.77m2', '\n\n\n', '\n\n\n\n追加\

['\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '6万円', '3500円', '12万円', '9万円', '2DK', '42.68m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '5万円', '-', '3万円', '2万円', 'ワンルーム', '32.71m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.3万円', '-', '5.3万円', '5.3万円', '2LDK', '53.4m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '5.7万円', '-', '5.7万円', '5.7万円', '2LDK', '53.4m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5万円', '-', '5万円', '-', '2DK', '52.99m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.5万円', '-', '5.5万円', '-', '2DK', '39.74m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.3万円', '2000円', '5.3万円', '5.3万円', '2DK

['\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t4階', '4.1万円', '-', '4.1万円', '-', '2K', '38.24m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '4.8万円', '1500円', '-', '4.8万円', '1K', '36.28m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '4.5万円', '-', '4.5万円', '-', '2K', '46.28m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '4.5万円', '-', '4.5万円', '-', '2K', '46.28m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5万円', '-', '6万円', '-', '2DK', '41m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '4.5万円', '-', '4.5万円', '-', 'ワンルーム', '25.67m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '5.8万円', '-', '5.8万円', '5.8万円', '2LDK', '56.83m2', '\n

['\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t4階', '6万円', '-', '-', '-', '1K', '20.16m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t3階', '3.5万円', '-', '-', '7万円', '2K', '33.66m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.2万円', '-', '4万円', '-', '1K', '24m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.5万円', '-', '5万円', '-', '2DK', '33m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.6万円', '-', '5万円', '-', '2DK', '33m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.8万円', '-', '3.8万円', '-', '3K', '56.3m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '3万円', '1500円', '-', '-', '1K', '22.64m2', 'パノラマ', '\n\n\n\n追加\n\n追加\n\n', 

['\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '3.4万円', '1000円', '-', '3.4万円', '1K', '23.4m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '3.5万円', '1000円', '-', '3.5万円', '1K', '23.4m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t1階', '4.1万円', '-', '-', '-', '1K', '26.44m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t3階', '4.1万円', '-', '-', '-', '1K', '26.44m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t4階', '4.1万円', '-', '-', '-', '1K', '26.44m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '7.8万円', '4000円', '15.6万円', '7.8万円', '1LDK', '50.23m2', '\n\n\n', '\n\n\n\n追加\n\n追加\n\n', '\n詳細を見る', '\n\xa0\n', '\n\n\n\n\n', '\r\n\t\t\t\t\t\t\t\t\t\t\t2階', '7.8万円', '4000円', '15.6万円', '7.8万円', '1LDK', '49